<a href="https://colab.research.google.com/github/SutapaSusovita/Nutrition_Agent/blob/main/Nutrion_AI_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response

# Setup

In [2]:
!pip install ibm-watsonx-ai langchain-ibm langgraph


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 15.2 MB/s eta 0:00:00
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.14.3-py3-none-any.whl size=7

In [3]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [35]:
import os
import getpass


# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [8]:
model_id = "ibm/granite-3-3-8b-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [9]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [28]:
# ✅ Directly set your project_id here
project_id = "46f752f6-63af-49e3-a01c-b579cde1e0e1"
space_id = os.getenv("SPACE_ID")

def get_credentials():
    return {
        "url": "https://us-south.ml.cloud.ibm.com",
        "apikey": getpass.getpass("Please enter your api key (hit enter): ")
    }

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"
    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()



Please enter your api key (hit enter): ··········


## Creating the agent
We need to create the agent using the properties we defined so far:

In [29]:
# ✅ Pass project_id explicitly to APIClient
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [30]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)


vector_index_id = "76e119fd-5004-430c-a3ee-8d17270ca3d1"

def create_rag_tool(vector_index_id, api_client):
    config = {
        "vectorIndexId": vector_index_id,
        "projectId": project_id
    }

    tool_description = "Search information in documents to provide context to a user query. Useful when asked to ground the answer in specific knowledge about Farming"

    return create_utility_agent_tool("RAGQuery", config, api_client, tool_description=tool_description)



def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")

    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }

    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )

        return results.get("output")

    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)

    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    tools.append(create_rag_tool(vector_index_id, client))

    config = {
        "maxResults": 10
    }
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("DuckDuckGo", config, client))
    config = {
        "maxResults": 5
    }
    tools.append(create_utility_agent_tool("Wikipedia", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("Weather", config, client))

    return tools

In [33]:
from langchain.prompts import ChatPromptTemplate
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

def create_agent(context):
    chat_model = create_chat_model()
    tools = create_tools(context)
    memory = MemorySaver()

    instructions = """You are a helpful nutrition AI agent.
Ask the user for their age, gender, weight, height, dietary preferences (e.g., vegetarian, keto), allergies, medical conditions (e.g., diabetes, PCOS), cultural preferences, and fitness goals (e.g., weight loss, muscle gain). Store the responses securely and use them to personalize future suggestions.
Generate a daily or weekly meal plan that aligns with the user's profile: dietary preferences, medical conditions, and fitness goals. Ensure nutritional balance (macros + micronutrients) and cultural relevance. Include breakfast, lunch, dinner, and optional snacks with calorie counts and prep details.
When a user inputs a food they dislike, are allergic to, or want a healthier version of, suggest smart alternatives. For example, if a user inputs "white rice", you might suggest "quinoa" or "cauliflower rice" and explain why the alternative is healthier.
Whenever a recommendation is made, provide a short, clear explanation such as:
- "This food is high in fiber, which supports digestion."
- "This alternative has a lower glycemic index, which is better for diabetes." Offer contextual, science-backed reasons to help the user make informed decisions.
If a user uploads a food photo or grocery label, extract the food item name using image recognition. Cross-reference the item with a nutrition database to determine if it's healthy based on the user's profile. Respond with nutritional info and suggestions if needed.
If a user says they disliked a meal, were still hungry, or experienced a symptom, adjust future recommendations accordingly. Ask follow-up questions like: "Would you prefer a lighter/heavier meal next time?" or "Should I avoid this ingredient in future plans?"
"""

    # Instead of passing 'prompt' directly to create_react_agent,
    # embed instructions into the model itself
    chat_model = chat_model.bind(system=instructions)

    agent = create_react_agent(
        chat_model,
        tools=tools,
        checkpointer=memory
    )
    return agent


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [34]:
from langchain.schema import HumanMessage, AIMessage

agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if message["role"] == "user":
            converted_messages.append(HumanMessage(content=message["content"]))
        elif message["role"] == "assistant":
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    {"messages": convert_messages(messages)},
    config={"configurable": {"thread_id": "42"}}
)

print_full_response = False

if print_full_response:
    print(generated_response)
else:
    if isinstance(generated_response, dict) and "messages" in generated_response:
        result = generated_response["messages"][-1].content
    else:
        result = str(generated_response)
    print(f"Agent: {result}")


Question: Hi, I want you to create a weekly vegetarian meal plan for me. I’m 28 years old, female, 60 kg, 165 cm tall. I have no allergies, but I have mild PCOS. I prefer Indian cuisine and my goal is weight loss.


Status code: 500, body: DDG detected an anomaly in the request, you are likely making requests too quickly.
Status code: 500, body: DDG detected an anomaly in the request, you are likely making requests too quickly.
Status code: 500, body: DDG detected an anomaly in the request, you are likely making requests too quickly.


Agent: To create a suitable meal plan for you, I'll need to consider your dietary preferences, health condition, and weight loss goals. Here's a simple weekly vegetarian meal plan focusing on Indian cuisine that could help with weight loss while managing mild PCOS. Please consult with a nutritionist or dietitian for personalized advice.

**Day 1:**
- **Breakfast:** Moong dal chilla (mung bean pancakes) with tomato chutney.
- **Lunch:** Quinoa salad with mixed vegetables (carrots, bell peppers, cucumbers) and lemon-tahini dressing.
- **Dinner:** Palak paneer (spinach and tofu) with whole wheat roti.

**Day 2:**
- **Breakfast:** Oats upma with peas and carrots.
- **Lunch:** Chickpea curry (chana masala) with brown rice.
- **Dinner:** Mixed vegetable curry with besan (chickpea flour) gravy, served with millet.

**Day 3:**
- **Breakfast:** Smoothie with spinach, banana, almond milk, and chia seeds.
- **Lunch:** Dal tadka with whole wheat naan.
- **Dinner:** Baingan bharta (roasted eggplant

# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  